In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━

In [ ]:
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


In [ ]:
import json
import os
from urllib.parse import urljoin

import requests
import torch
import transformers
from accelerate import init_empty_weights
from bs4 import BeautifulSoup
from datasets import Dataset

In [ ]:
import os
os.environ['WANDB_DISABLED']="true"

In [ ]:
BASE_URL = "https://kubernetes.io/docs/"
CACHE_FILE = "k8s_docs.json"


def get_all_kubernetes_pages():
    response = requests.get(BASE_URL)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        links = soup.find_all("a", href=True)

        pages = [urljoin(BASE_URL, link['href']) for link in links if link['href'].startswith('/docs/')]
        return list(set(pages))
    else:
        print("Failed to retrieve Kubernetes docs page.")
        return []

def scrape_kubernetes_docs(force_refresh=False):
    if os.path.exists(CACHE_FILE) and not force_refresh:
        print("Loading cached Kubernetes documentation...")
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)

    print("Scraping Kubernetes documentation...")
    all_pages = get_all_kubernetes_pages()
    dataset = []

    for url in all_pages:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            page_text = soup.get_text(separator="\n", strip=True)

            dataset.append({"question": f"What is discussed on {url}?", "answer": page_text})
            print(f"Scraped {url}")
        else:
            print(f"Failed to scrape {url}")

    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(dataset, f, indent=4)

    return dataset


def prepare_finetuning_dataset():
    dataset = scrape_kubernetes_docs()

    hf_dataset = Dataset.from_list(dataset)
    hf_dataset = hf_dataset.train_test_split(test_size=0.1)

    train_eval = hf_dataset["train"].train_test_split(test_size=0.111)
    hf_dataset["train"] = train_eval["train"]
    hf_dataset["eval"] = train_eval["test"]
    return hf_dataset

dataset = prepare_finetuning_dataset()
print(dataset)
df = pd.DataFrame(dataset['train'])
df

Loading cached Kubernetes documentation...
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 616
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 78
    })
    eval: Dataset({
        features: ['question', 'answer'],
        num_rows: 78
    })
})


,question,answer
0,What is discussed on https://kubernetes.io/doc...,Viewing Site Analytics | Kubernetes\nViewing S...
1,What is discussed on https://kubernetes.io/doc...,Access Clusters Using the Kubernetes API | Kub...
2,What is discussed on https://kubernetes.io/doc...,RuntimeClass | Kubernetes\nRuntimeClass | Kube...
3,What is discussed on https://kubernetes.io/doc...,kubectl create service externalname | Kubernet...
4,What is discussed on https://kubernetes.io/doc...,Communication between Nodes and the Control Pl...
...,...,...
611,What is discussed on https://kubernetes.io/doc...,Extend Kubernetes | Kubernetes\nExtend Kuberne...
612,What is discussed on https://kubernetes.io/doc...,Storage Capacity | Kubernetes\nStorage Capacit...
613,What is discussed on https://kubernetes.io/doc...,kubeadm upgrade | Kubernetes\nkubeadm upgrade ...
614,What is discussed on https://kubernetes.io/doc...,Managing Secrets using kubectl | Kubernetes\nM...


In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
model_name='microsoft/phi-2'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('question', 'answer')
    Then concatenate them using two newline characters.

    :param sample: Sample dictionary containing 'question' and 'answer'.
    """
    INTRO_BLURB = "Below is a question about a Kubernetes documentation page. Provide a detailed response summarizing its content."
    INSTRUCTION_KEY = "### Question:"
    RESPONSE_KEY = "### Answer:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['question']}" if sample.get("question") else None
    response = f"{RESPONSE_KEY}\n{sample['answer']}" if sample.get("answer") else None
    end = f"{END_KEY}"

    # Combine only non-empty parts
    parts = [part for part in [blurb, instruction, response, end] if part]
    formatted_prompt = "\n\n".join(parts)

    # Add the formatted text back to the sample
    sample["text"] = formatted_prompt

    return sample


In [ ]:
from functools import partial

def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['question', 'answer'],
    )

    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) <= max_length)

    dataset = dataset.shuffle(seed=seed)

    return dataset

In [ ]:
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['eval'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/616 [00:00<?, ? examples/s]

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

Filter:   0%|          | 0/616 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Filter:   0%|          | 0/78 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 616
})

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
original_model = prepare_model_for_kbit_training(original_model)

config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [ ]:
import transformers

output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=20,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
peft_trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=20, training_loss=1.1840206146240235, metrics={'train_runtime': 990.7698, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.02, 'total_flos': 2624307226214400.0, 'train_loss': 1.1840206146240235, 'epoch': 0.12987012987012986})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/peft-dialogue-summary-training-1739166883/checkpoint-20",torch_dtype=torch.float16,is_trainable=False)

In [ ]:
def chat():
    """Deploys the fine-tuned chatbot to answer Kubernetes queries."""
    model, tokenizer = ft_model, eval_tokenizer
    model.eval()

    print("\nKubernetes Chatbot Ready! Type 'exit' to quit.")
    while True:
        user_query = input("\nAsk about Kubernetes: ")
        if user_query.lower() == "exit":
            break

        input_tokens = tokenizer(user_query, return_tensors="pt").to("cuda")
        with torch.no_grad():
            output_tokens = model.generate(**input_tokens, max_new_tokens=500)

        response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        print("\nChatbot:", response)

In [ ]:
chat()